# experiment

> Notebook where the training experiments take place

In [ ]:
#| default_exp experiment

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *
from fastcore.utils import *

In [ ]:
#| hide
import os

if os.getenv("COLAB_RELEASE_TAG"):
   print("Running in Colab")       
   from google.colab import drive
   drive.mount('/content/drive')
   %cd /content/drive/MyDrive/GitHub/dm-4-cat
   %pip install wandb
else:
   print("NOT in Colab")

NOT in Colab


In [ ]:
#| export
import wandb

from birdclef.trainer import train

These are the variables that must be set to start an experiment:

1. **project**: The name of the wandb project where the training, evaluation and test results will be logged and stored.

2. **entity**: The wandb entity associated with the project.

3. **sweep_name**: The name given to the sweep configuration, which defines the hyperparameter search setup for an experiment. It's used to organize and categorize different hyperparameter tuning runs.

4. **method**: The method or strategy used for hyperparameter tuning. In this case, 'random' suggests that hyperparameters will be randomly chosen from the specified ranges or values during the sweep.

5. **n_runs**: The number of runs or iterations that will be performed during the hyperparameter sweep. Each run involves training the model with a specific set of hyperparameters.

6. **run_name**: The name given to each individual run or iteration of the experiment. It helps identify and differentiate between different runs, providing a meaningful label for tracking and analysis.

7. **device**: The computational device (e.g., 'cpu', 'cuda') on which the training and evaluation of the model will be performed.

8. **train_key**: Key or identifier used to access the training dataset. Refer to `get_dataset` for info about available keys.

9. **val_key**: Key or identifier used to access the validation dataset. Refer to `get_dataset` for info about available keys.

10. **test_key**: Key or identifier used to access the test dataset. Refer to `get_dataset` for info about available keys.

11. **batch_size**: The number of samples in each mini-batch during training. It affects the efficiency of the training process and the model's ability to generalize.

12. **num_workers**: The number of worker threads used to load data in parallel during training. It can help speed up the data loading process.

13. **pin_memory**: A boolean indicating whether to pin memory for faster data transfer to the GPU. This is often beneficial when using a GPU for training.

14. **model_key**: Key or identifier used to specify the model architecture to be used for training. Refer to `get_model` for info about available keys.

15. **optimizer_key**: Key or identifier used to specify the optimizer to be used during the training process. Refer to `get_optimizer` for info about available keys.

16. **learning_rate**: A list of learning rates to be used by the optimizer during training. Learning rate is a crucial hyperparameter affecting the convergence and performance of the model.

17. **loss_key**: Key or identifier used to specify the loss function to be used during training. Refer to `get_loss_func` for info about available keys.

18. **metric**: The metric used to evaluate the model's performance. This metric is used to compare and choose the best model in a single run. Refer to `compute_metrics` for info about available metrics.

19. **epochs**: The number of epochs or complete passes through the training dataset during the training process. One epoch is a single pass through the entire training dataset.

In [ ]:
project = 'bird-clef-cjavelanda'
entity = '4projects'

sweep_name = 'test' 
method = 'random'
n_runs = 1

run_name = 'test3' 
device = 'cpu' 
train_key = 'train_simple' 
val_key = 'val_simple' 
test_key = 'test_simple'
batch_size = 16
num_workers = 2
pin_memory = False
model_key = 'simple_efficient_net_v2_s' 
optimizer_key = 'adamw' 
learning_rate = [0.00005] 
loss_key = 'bce'
metric = 'f1'
epochs = 3

Creating the experiment configuration as dict.

In [ ]:
sweep_config = {
    'name': sweep_name,
    'method': method,
    'parameters': {
        'run_name': {
            'value': run_name
        },
        'device': {
            'value': device
        },
        'train_key': {
            'value': train_key
        },
        'train_kwargs': {
            'parameters': {
                'batch_size': {
                    'value': batch_size
                },
                'shuffle': {
                    'value': True
                },
                'num_workers': {
                    'value': num_workers 
                },
                'pin_memory': {
                    'value': pin_memory
                }
            }
        },
        'val_key': {
            'value': val_key
        },
        'test_key': {
            'value': test_key
        },
        'val_kwargs': {
            'parameters': {
                'batch_size': {
                    'value': batch_size
                },
                'shuffle': {
                    'value': False
                },
                'num_workers': {
                    'value': num_workers 
                },
                'pin_memory': {
                    'value': pin_memory
                }
            }
        },
        'model_key': {
            'value': model_key
        },
        'optimizer_key': {
            'value': optimizer_key
        },
        'optimizer_kwargs': {
            'parameters': {
                'lr': {
                    'values': learning_rate
                },
            }
        },
        'loss_key': {
            'value': loss_key
        },
        'metric': {
            'value': metric
        },
        'epochs': {
            'value': epochs
        }
    }  
}

Running a sweep.

In [ ]:
#|eval: false
#|output: false
sweep_id = wandb.sweep(sweep_config, project=project, entity=entity)
wandb.agent(
    sweep_id,
    train,
    count=n_runs)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: b0v95stj
Sweep URL: https://wandb.ai/4projects/bird-clef-cjavelanda/sweeps/b0v95stj


wandb: Agent Starting Run: mwjys6a2 with config:
wandb: 	device: cpu
wandb: 	epochs: 3
wandb: 	loss_key: bce
wandb: 	metric: f1
wandb: 	model_key: simple_efficient_net_v2_s
wandb: 	optimizer_key: adamw
wandb: 	optimizer_kwargs: {'lr': 5e-05}
wandb: 	run_name: test3
wandb: 	test_key: test_simple
wandb: 	train_key: train_simple
wandb: 	train_kwargs: {'batch_size': 16, 'num_workers': 2, 'pin_memory': False, 'shuffle': True}
wandb: 	val_key: val_simple
wandb: 	val_kwargs: {'batch_size': 16, 'num_workers': 2, 'pin_memory': False, 'shuffle': False}
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chavelanda (4projects). Use `wandb login --relogin` to force relogin


Training epoch 0


100%|██████████| 53/53 [14:47<00:00, 16.74s/it]


...

	Finished training. Starting validation


100%|██████████| 21/21 [01:34<00:00,  4.50s/it]


	Finshed validation
	Metrics logged to wandb
	f1 in the validation set has improved!
	Testing with best model


100%|██████████| 21/21 [01:26<00:00,  4.12s/it]


train/accuracy,▁▃▅▄▃▅▂▄▄▄▂▃▄▃▅▅▄▁▅▃▅▃▄▁▂▄▄▂▃▄▃▄█▂▅▄▄▅▅▂
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/f1,▁▂▅▃▂▅▂▄▃▄▂▂▄▃▅▆▄▁▆▃▆▃▅▁▁▄▄▂▄▅▃▄█▃▄▄▄▅▅▃
train/loss,▄▃▇▅▅▃▆▃▆▂▆▆▃▄▆▂▃▇▂█▆▇▃▆▆▃▄▇▅▂▅▃▁▃▅▅▃▃▃▃
train/precision,▂▂▅▃▅▇▂▄▃▄▅▂▅▃▅▆▄▁▇▂▅▄▅▁▁█▄▃▅▆▃▃█▅▄▅▄▆▅▃
train/recall,▁▃▅▄▃▅▂▄▄▄▂▃▄▃▅▅▄▁▅▃▅▃▄▁▂▄▄▂▃▄▃▄█▂▅▄▄▅▅▂
train/step_ct,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val/accuracy,▁▃█
val/epoch,▁▅█
val/example_ct,▁▅█


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()